# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = 'data/song_data/A/A/C/TRAACCG128F92E8A55.json'

In [6]:
df = pd.read_json(filepath,lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR5KOSW1187FB35FF4,49.80388,15.47491,Dubai UAE,Elena,SOZCTXZ12AB0182364,Setanta matins,269.58322,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_columns=['song_id','title','artist_id','year','duration']
song_data = df[song_columns].values[0]
song_data

array(['SOZCTXZ12AB0182364', 'Setanta matins', 'AR5KOSW1187FB35FF4', 0,
       269.58322], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_columns=['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']
artist_data = df[artist_columns].values[0]
artist_data

array(['AR5KOSW1187FB35FF4', 'Elena', 'Dubai UAE', 49.80388, 15.47491],
      dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

In [12]:
filepath = log_files[20]

In [13]:
df = pd.read_json(filepath,lines=True)
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Elena,Logged In,Lily,F,5,Koch,269.58322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Setanta matins,200,1542837407796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
1,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,741,None,307,1542760054796,None,
2,None,Logged In,Theodore,M,1,Smith,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540306e+12,741,None,200,1542760086796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52
3,None,Logged Out,None,None,0,None,NaN,paid,None,GET,Home,NaN,774,None,200,1542761399796,None,
4,None,Logged Out,None,None,1,None,NaN,paid,None,GET,Home,NaN,774,None,200,1542761485796,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,Martina Topley Bird,Logged In,Lily,F,34,Koch,177.08363,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Poison,200,1542844025796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
513,Ben Jelen,Logged In,Lily,F,35,Koch,226.55955,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Come On (Album Version),200,1542844202796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
514,Florence + The Machine,Logged In,Theodore,M,0,Smith,131.31710,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540306e+12,796,Kiss With A Fist,200,1542844308796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52
515,3 Doors Down,Logged In,Lily,F,36,Koch,237.13914,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Here Without You,200,1542844428796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
Filter = (df['page']=='NextSong')
df = df[Filter]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Elena,Logged In,Lily,F,5,Koch,269.58322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Setanta matins,200,1542837407796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
7,Facto Delafe y las flores azules,Logged In,Tegan,F,4,Levine,315.81995,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Enero en la playa,200,1542761878796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
8,Kings Of Leon,Logged In,Kate,F,0,Harrell,204.27710,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Manhattan,200,1542761921796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
9,Franz Ferdinand,Logged In,Kate,F,1,Harrell,204.12036,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Michael,200,1542762125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
10,Blue October,Logged In,Tegan,F,5,Levine,272.32608,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Drilled A Wire Through My Cheek,200,1542762193796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80


In [15]:
t = pd.to_datetime(df['ts'] , unit='ms')
t.head()

0    2018-11-21 21:56:47.796
7    2018-11-21 00:57:58.796
8    2018-11-21 00:58:41.796
9    2018-11-21 01:02:05.796
10   2018-11-21 01:03:13.796
Name: ts, dtype: datetime64[ns]

In [16]:
get=t.dt
time_data = (t,get.hour,get.day,get.week,get.month,get.year,get.day_name())
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [17]:
time_dict={}
for i in range(len(time_data)):
    time_dict[column_labels[i]]=time_data[i]

time_df = pd.DataFrame.from_dict(time_dict)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-21 21:56:47.796,21,21,47,11,2018,Wednesday
7,2018-11-21 00:57:58.796,0,21,47,11,2018,Wednesday
8,2018-11-21 00:58:41.796,0,21,47,11,2018,Wednesday
9,2018-11-21 01:02:05.796,1,21,47,11,2018,Wednesday
10,2018-11-21 01:03:13.796,1,21,47,11,2018,Wednesday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_columns=['userId', 'firstName', 'lastName', 'gender', 'level']
user_df = df[user_columns]
user_df.head()

,userId,firstName,lastName,gender,level
0,15,Lily,Koch,F,paid
7,80,Tegan,Levine,F,paid
8,97,Kate,Harrell,F,paid
9,97,Kate,Harrell,F,paid
10,80,Tegan,Levine,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    row.song
    row.artist
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    start_time = pd.to_datetime(row.ts , unit='ms')
    songplay_data = (index,start_time,int(row.userId),row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.